In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import tensorflow as tf

from thesis import pqmf
from ddsp.colab.colab_utils import (
    auto_tune, get_tuning_factor, download,
    play, record, specplot, upload, audio_bytes_to_np,
    DEFAULT_SAMPLE_RATE)
import numpy as np
import IPython

def play_audio(audio):
    audio = np.array(audio)
    audio = np.squeeze(audio)
    IPython.display.display(IPython.display.Audio(audio, rate=16000))

import ddsp
from ddsp.losses import SpectralLoss
import einops

import gin
gin.enter_interactive_mode()

def get_loss(target_audio, audio):
    loss_class = SpectralLoss()
    loss = loss_class(tf.convert_to_tensor(target_audio), tf.convert_to_tensor(audio))
    return float(loss)

In [ ]:
from thesis import rave

In [ ]:
synthesizer = ddsp.synths.FilteredNoise(n_samples=64000)
filter_bank_magnitudes = tf.zeros((3, 16000, 100), dtype=tf.float32) + 1.0
filter_bank_magnitudes = tf.linspace(10, -10, 100)
filter_bank_magnitudes = einops.repeat(filter_bank_magnitudes, "a -> b samples a", b=3, samples=16000)

magnitudes = [tf.sin(tf.linspace(0.0, w, 100)) for w in np.linspace(8.0, 80.0, 250)]
magnitudes = 0.5 * tf.stack(magnitudes)**4.0
magnitudes = magnitudes[tf.newaxis, :, :]

output = synthesizer(magnitudes)
specplot(output[0] * 100)

In [ ]:
play_audio(output[0])

In [ ]:
p = rave.MultibandFilteredNoise(n_bands=4)
fbm = einops.rearrange(magnitudes, "b samples (bands filter_banks) -> b samples filter_banks bands", bands=4)
y = p(fbm)

In [ ]:
y.shape

In [ ]:
specplot(y[0,:,0])

In [ ]:
play_audio(y[0,:,0])

In [ ]:
play_audio(y[0,:,1])

In [ ]:
play_audio(y[0,:,2])

In [ ]:
play_audio(y[0,:,3])